<a href="https://colab.research.google.com/github/andygma567/LLM-experiments/blob/main/few_shot_prompt_Falcon_7B_LangChain_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sat Jun 24 13:41:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%bash
pip -q install bitsandbytes accelerate # required for efficient loading / inference
# !pip -q install git+https://github.com/huggingface/transformers # need to install from github
pip install -q transformers # version 4.30.2
pip install -q gradio
pip install -q einops # required for Falcon
pip -q install langchain

pip -q install xformers # required for Falcon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import transformers
import torch

In [ ]:
# LangChain stuff
import langchain
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

In [ ]:
import warnings
# warnings.filterwarnings("ignore")

# Load the model


In [ ]:
HF_ckpt = "tiiuae/falcon-7b-instruct"
drive_ckpt = "tiiuae/falcon-7b-instruct"

In [ ]:
# Test using AutoModel... it didn't work for me earlier, but maybe I had a typo?
# Test using 8bit

# It all works!
%%time
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

tokenizer = AutoTokenizer.from_pretrained(HF_ckpt)
model = AutoModelForCausalLM.from_pretrained(
    HF_ckpt,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    load_in_8bit=True, # I forgot if 8bit is faster than float16... I think it is but...idk
    device_map="auto",
    offload_folder="offload", # for some reason the offload folder doesn't work for this model
)
# without an offload folder, I often use a high-RAM shape of colab...

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 28.3 s, sys: 38.4 s, total: 1min 6s
Wall time: 4min 17s


# Load a pipeline

In [ ]:
%%time
pipeline = transformers.pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
)
generation_config = GenerationConfig(
    max_new_tokens=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
sequences = pipeline(
   "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron:",
   generation_config=generation_config
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNormForC

Result: Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.
Daniel: Hello, Girafatron!
Girafatron: Hi, Daniel. I'm so glad you found me. How are you doing today?
Daniel: I'm doing great, thanks. So, how have you been?
Girafatron: Oh, I'm just fine. I'm in my element, surrounded by giraffes. Life is good.
Daniel: That sounds nice. What's been going on with the giraffes lately?
Girafatron: Oh, just the usual. They're all eating lots and sleeping a lot. I'm the one keeping an eye on them and making sure they're safe and sound.
Daniel: That's cool. Have you seen any new animals around?
Girafatron: Actually, there's something new. I think I spotted a few meerkats hanging out in the distance.
Daniel: That sounds interesting. Have you tried to approach them?
Girafatron:
CPU times: user 1min 8s, sys: 595 ms, total: 1min 9s
Wall time: 1min 14s


In [ ]:
HF_checkpoint = "tiiuae/falcon-7b-instruct"
# tokenizer = AutoTokenizer.from_pretrained(HF_checkpoint)

# Test using LangChain Tools


## Create a wrapper for Falcon

In [ ]:
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any

class LocalFalcon(LLM):
    generation_config: GenerationConfig

    @property
    def _llm_type(self) -> str:
        return "custom"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        # pipeline returns a list of dictionaries (due to batching)
        # each dictionary has one key "generated_text"
        batch_output = pipeline(
            prompt,
            generation_config=self.generation_config
        )

        output_dict = batch_output[0]
        return output_dict["generated_text"]

In [ ]:
generation_config = GenerationConfig(
                      max_new_tokens=1024,
                      do_sample=True,
                      top_k=10,
                      num_return_sequences=1,
                      eos_token_id=tokenizer.eos_token_id,
                    )
wrapped_llm = LocalFalcon(generation_config=generation_config)
print(wrapped_llm)

LocalFalcon
Params: {}


In [ ]:
# test
%%time
question = "What is a Falcon?"
print(wrapped_llm(question))

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/IPython/core/magics/execution.py:1340 in time            │
│                                                                                                  │
│   1337 │   │   │   │   # multi-line %%time case                                                  │
│   1338 │   │   │   │   if expr_val is not None:                                                  │
│   1339 │   │   │   │   │   code_2 = self.shell.compile(expr_val, source, 'eval')                 │
│ ❱ 1340 │   │   │   │   │   out = eval(code_2, glob, local_ns)                                    │
│   1341 │   │   │   except:                                                                       │
│   1342 │   │   │   │   self.shell.showtraceback()                                                │
│   1343 │   │   │   │   return                                                                    │
│ in <module>:2                                                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/llms/base.py:338 in __call__                   │
│                                                                                                  │
│   335 │   │   │   │   "`generate` instead."                                                      │
│   336 │   │   │   )                                                                              │
│   337 │   │   return (                                                                           │
│ ❱ 338 │   │   │   self.generate([prompt], stop=stop, callbacks=callbacks, **kwargs)              │
│   339 │   │   │   .generations[0][0]                                                             │
│   340 │   │   │   .text                                                                          │
│   341 │   │   )                                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/llms/base.py:206 in generate                   │
│                                                                                                  │
│   203 │   │   │   │   )                                                                          │
│   204 │   │   │   except (KeyboardInterrupt, Exception) as e:                                    │
│   205 │   │   │   │   run_manager.on_llm_error(e)                                                │
│ ❱ 206 │   │   │   │   raise e                                                                    │
│   207 │   │   │   run_manager.on_llm_end(output)                                                 │
│   208 │   │   │   if run_manager:                                                                │
│   209 │   │   │   │   output.run = RunInfo(run_id=run_manager.run_id)                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/llms/base.py:198 in generate                   │
│                                                                                                  │
│   195 │   │   │   )                                                                              │
│   196 │   │   │   try:                                                                           │
│   197 │   │   │   │   output = (                                                                 │
│ ❱ 198 │   │   │   │   │   self._generate(                                                        │
│   199 │   │   │   │   │   │   prompts, stop=stop, run_manager=run_manager, **kwargs              │
│   200 │   │   │   │   │   )                                

## Test a Python Agent

References: https://learn.deeplearning.ai/langchain/lesson/7/agents

In [ ]:
# https://python.langchain.com/en/latest/modules/agents/toolkits/examples/python.html
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL

# If I want to use PythonREPL directly I think
# from langchain.utilities import PythonREPL

In [ ]:
agent_executor = create_python_agent(
    llm=wrapped_llm,
    tool=PythonREPLTool(),
    verbose=True,
    handle_parsing_errors=True,
)

# setting handle_parsing_errors="Check your output and make sure it conforms!", # I think this reruns the chain with this prompt?

In [ ]:
print(agent_executor.agent.llm_chain.prompt)

input_variables=['input', 'agent_scratchpad'] output_parser=None partial_variables={} template='You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an error, debug your code and try again.\nOnly use the output of your code to answer the question. \nYou might know the answer without running any code, but you should still run the code to get the answer.\nIf it does not seem like you can write code to answer the question, just return "I don\'t know" as the answer.\n\n\nPython_REPL: A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Python_REPL]\nAction Input: the input to the a

In [ ]:
print(agent_executor.agent.llm_chain.prompt.template)

You are an agent designed to write and execute python code to answer questions.
You have access to a python REPL, which you can use to execute python code.
If you get an error, debug your code and try again.
Only use the output of your code to answer the question. 
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.


Python_REPL: A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Python_REPL]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times

In [ ]:
%%time
print(pipeline("What is 17 times 10?", max_new_tokens=200)[0]["generated_text"])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


What is 17 times 10?
17 times 10 is equal to 170.
CPU times: user 2.4 s, sys: 0 ns, total: 2.4 s
Wall time: 2.59 s


In [ ]:
%%time
agent_executor.run("What is 17 times 10?")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new  chain...


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/IPython/core/magics/execution.py:1327 in time            │
│                                                                                                  │
│   1324 │   │   if mode=='eval':                                                                  │
│   1325 │   │   │   st = clock2()                                                                 │
│   1326 │   │   │   try:                                                                          │
│ ❱ 1327 │   │   │   │   out = eval(code, glob, local_ns)                                          │
│   1328 │   │   │   except:                                                                       │
│   1329 │   │   │   │   self.shell.showtraceback()                                                │
│   1330 │   │   │   │   return                                                                    │
│ in <module>:1                                                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:290 in run                      │
│                                                                                                  │
│   287 │   │   if args and not kwargs:                                                            │
│   288 │   │   │   if len(args) != 1:                                                             │
│   289 │   │   │   │   raise ValueError("`run` supports only one positional argument.")           │
│ ❱ 290 │   │   │   return self(args[0], callbacks=callbacks, tags=tags)[_output_key]              │
│   291 │   │                                                                                      │
│   292 │   │   if kwargs and not args:                                                            │
│   293 │   │   │   return self(kwargs, callbacks=callbacks, tags=tags)[_output_key]               │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:166 in __call__                 │
│                                                                                                  │
│   163 │   │   │   )                                                                              │
│   164 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   165 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 166 │   │   │   raise e                                                                        │
│   167 │   │   run_manager.on_chain_end(outputs)                                                  │
│   168 │   │   final_outputs: Dict[str, Any] = self.prep_outputs(                                 │
│   169 │   │   │   inputs, outputs, return_only_outputs                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:160 in __call__                 │
│                                                                                                  │
│   157 │   │   )                                                                                  │
│   158 │   │   try:                                                                               │
│   159 │   │   │   outputs = (                                                                    │
│ ❱ 160 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   161 │   │   │   │   if new_arg_supported                                                       │
│   162 │   │   │   │   else self._call(inputs)              

In [ ]:
%%time
agent_executor.run("Write a Python script that prints 'Hello, world!'")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new  chain...


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/IPython/core/magics/execution.py:1327 in time            │
│                                                                                                  │
│   1324 │   │   if mode=='eval':                                                                  │
│   1325 │   │   │   st = clock2()                                                                 │
│   1326 │   │   │   try:                                                                          │
│ ❱ 1327 │   │   │   │   out = eval(code, glob, local_ns)                                          │
│   1328 │   │   │   except:                                                                       │
│   1329 │   │   │   │   self.shell.showtraceback()                                                │
│   1330 │   │   │   │   return                                                                    │
│ in <module>:1                                                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:290 in run                      │
│                                                                                                  │
│   287 │   │   if args and not kwargs:                                                            │
│   288 │   │   │   if len(args) != 1:                                                             │
│   289 │   │   │   │   raise ValueError("`run` supports only one positional argument.")           │
│ ❱ 290 │   │   │   return self(args[0], callbacks=callbacks, tags=tags)[_output_key]              │
│   291 │   │                                                                                      │
│   292 │   │   if kwargs and not args:                                                            │
│   293 │   │   │   return self(kwargs, callbacks=callbacks, tags=tags)[_output_key]               │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:166 in __call__                 │
│                                                                                                  │
│   163 │   │   │   )                                                                              │
│   164 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   165 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 166 │   │   │   raise e                                                                        │
│   167 │   │   run_manager.on_chain_end(outputs)                                                  │
│   168 │   │   final_outputs: Dict[str, Any] = self.prep_outputs(                                 │
│   169 │   │   │   inputs, outputs, return_only_outputs                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:160 in __call__                 │
│                                                                                                  │
│   157 │   │   )                                                                                  │
│   158 │   │   try:                                                                               │
│   159 │   │   │   outputs = (                                                                    │
│ ❱ 160 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   161 │   │   │   │   if new_arg_supported                                                       │
│   162 │   │   │   │   else self._call(inputs)              

In [ ]:
%%time
customer_list = [["Harrison", "Chase"],
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"],
                 ["Geoff","Fusion"],
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]
agent_executor.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new  chain...


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 9>:9                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:290 in run                      │
│                                                                                                  │
│   287 │   │   if args and not kwargs:                                                            │
│   288 │   │   │   if len(args) != 1:                                                             │
│   289 │   │   │   │   raise ValueError("`run` supports only one positional argument.")           │
│ ❱ 290 │   │   │   return self(args[0], callbacks=callbacks, tags=tags)[_output_key]              │
│   291 │   │                                                                                      │
│   292 │   │   if kwargs and not args:                                                            │
│   293 │   │   │   return self(kwargs, callbacks=callbacks, tags=tags)[_output_key]               │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:166 in __call__                 │
│                                                                                                  │
│   163 │   │   │   )                                                                              │
│   164 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   165 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 166 │   │   │   raise e                                                                        │
│   167 │   │   run_manager.on_chain_end(outputs)                                                  │
│   168 │   │   final_outputs: Dict[str, Any] = self.prep_outputs(                                 │
│   169 │   │   │   inputs, outputs, return_only_outputs                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:160 in __call__                 │
│                                                                                                  │
│   157 │   │   )                                                                                  │
│   158 │   │   try:                                                                               │
│   159 │   │   │   outputs = (                                                                    │
│ ❱ 160 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   161 │   │   │   │   if new_arg_supported                                                       │
│   162 │   │   │   │   else self._call(inputs)                                                    │
│   163 │   │   │   )                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/agents/agent.py:987 in _call                   │
│                                                                                                  │
│    984 │   │   start_time = time.time()                                                          │
│    985 │   │   # We now enter the agent loop (until it returns something).                       │
│    986 │   │   while self._should_continue(iterations, time_elapsed):                            │
│ ❱  987 │   │   │   next_step_output = self._take_next_step(                                      │
│    988 │   │   │   │   name_to_tool_map,                   

In [ ]:
langchain.debug=True
agent_executor.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""")
langchain.debug=False

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:LocalFalcon] Entering LLM run with input:
{
  "prompts": [
    "You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you 

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:2                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:290 in run                      │
│                                                                                                  │
│   287 │   │   if args and not kwargs:                                                            │
│   288 │   │   │   if len(args) != 1:                                                             │
│   289 │   │   │   │   raise ValueError("`run` supports only one positional argument.")           │
│ ❱ 290 │   │   │   return self(args[0], callbacks=callbacks, tags=tags)[_output_key]              │
│   291 │   │                                                                                      │
│   292 │   │   if kwargs and not args:                                                            │
│   293 │   │   │   return self(kwargs, callbacks=callbacks, tags=tags)[_output_key]               │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:166 in __call__                 │
│                                                                                                  │
│   163 │   │   │   )                                                                              │
│   164 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   165 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 166 │   │   │   raise e                                                                        │
│   167 │   │   run_manager.on_chain_end(outputs)                                                  │
│   168 │   │   final_outputs: Dict[str, Any] = self.prep_outputs(                                 │
│   169 │   │   │   inputs, outputs, return_only_outputs                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:160 in __call__                 │
│                                                                                                  │
│   157 │   │   )                                                                                  │
│   158 │   │   try:                                                                               │
│   159 │   │   │   outputs = (                                                                    │
│ ❱ 160 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   161 │   │   │   │   if new_arg_supported                                                       │
│   162 │   │   │   │   else self._call(inputs)                                                    │
│   163 │   │   │   )                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/agents/agent.py:987 in _call                   │
│                                                                                                  │
│    984 │   │   start_time = time.time()                                                          │
│    985 │   │   # We now enter the agent loop (until it returns something).                       │
│    986 │   │   while self._should_continue(iterations, time_elapsed):                            │
│ ❱  987 │   │   │   next_step_output = self._take_next_step(                                      │
│    988 │   │   │   │   name_to_tool_map,                   

In [ ]:
langchain.debug=False

# Test a few-shot prompt for using Python



In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [ ]:
examples = [
  {
    "question": "Find out how much 2 plus 2 is.",
    "answer":
"""
Thought: I must use the Python shell to calculate 2 + 2
Action: Python REPL
Action Input:
2 + 2
Observation: 4

Thought: I now know the answer
Final Answer: 4
"""
  },
  {
    "question": "You have a variable age in your scope. If it's greater or equal than 21, say OK. Else, say Nay.",
    "answer":
"""
Thought: I should write an if/else block in the Python shell.
Action: Python REPL
Action Input:
if age >= 21:
    print("OK")  # this line has four spaces at the beginning
else:
    print("Nay")  # this line has four spaces at the beginning

Observation: OK
Thought: I now know the answer
Final Answer: I have executed the task successfully.
"""
  },
  {
    "question": "Write and execute a script that sleeps for 2 seconds and prints 'Hello, World'",
    "answer":
"""
Thought: I should import the sleep function.
Action: Python REPL
Action Input:
from time import sleep
Observation:

Thought: I should call the sleep function passing 2 as parameter
Action: Python REPL
Action Input:
from time import sleep
sleep(2)
Observation:

Thought: I should use the 'print' function to print 'Hello, World'
Action: Python REPL
Action Input:
from time import sleep
sleep(2)
print('Hello, World')
Observation: Hello, World

Thought: I now finished the script
Final Answer: I executed the following script successfully:

from time import sleep
sleep(2)
print('Hello, World')
"""
  },
]

In [ ]:
template_string="""Example:
Question: {question}
{answer}
"""

In [ ]:
example_prompt = PromptTemplate(input_variables=["question", "answer"], template=template_string)
print(example_prompt.format(**examples[0]))

Example:
Question: Find out how much 2 plus 2 is.

Thought: I must use the Python shell to calculate 2 + 2
Action: Python REPL
Action Input:
2 + 2
Observation: 4

Thought: I now know the answer
Final Answer: 4




In [ ]:
suffix_string="""
Additional Hints:
1. If an error thrown along the way, try to understand what happened and retry with a new code version that fixes the error.
2. DO NOT IGNORE ERRORS.
3. If an object does not have an attribute, call dir(object) to debug it.
4. SUPER IMPORTANT: ALWAYS respect the indentation in Python. Loops demand an idendentation. For example:

for i in range(10):
    print(i)  # this line has four spaces at the beginning

Same for ifs:

if True:
    print("hello")  # this line has four spaces at the beginning

An error be thrown because of the indentation, something like...  "expected an indented block after 'for' statement on line..."

To fix, make sure to indent the lines!

5. Do not use \ in variable names, otherwise you'll see the syntax error "unexpected character after line continuation character..."
6. If the variable is not defined, use vars() to see the defined variables.
7. Do not repeat the same statement twice without a new reason.
8. NEVER print the HTML directly.

Now begin for real!

Question: {input}
"""

In [ ]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix=suffix_string,
    input_variables=["input"]
)

In [ ]:
print(prompt.format(input="Write a Python script that prints 'Hello, world!'"))

Example:
Question: Find out how much 2 plus 2 is.

Thought: I must use the Python shell to calculate 2 + 2
Action: Python REPL
Action Input:
2 + 2
Observation: 4

Thought: I now know the answer
Final Answer: 4



Example:
Question: You have a variable age in your scope. If it's greater or equal than 21, say OK. Else, say Nay.

Thought: I should write an if/else block in the Python shell.
Action: Python REPL
Action Input:
if age >= 21:
    print("OK")  # this line has four spaces at the beginning
else:
    print("Nay")  # this line has four spaces at the beginning

Observation: OK
Thought: I now know the answer
Final Answer: I have executed the task successfully.



Example:
Question: Write and execute a script that sleeps for 2 seconds and prints 'Hello, World'

Thought: I should import the sleep function.
Action: Python REPL
Action Input:
from time import sleep
Observation:

Thought: I should call the sleep function passing 2 as parameter
Action: Python REPL
Action Input:
sleep(2)
Obs

In [ ]:
%%time
question1 = "Write a Python script that prints 'Hello, world!'"
agent_executor.run(prompt.format(input=question1))

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new  chain...


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/IPython/core/magics/execution.py:1340 in time            │
│                                                                                                  │
│   1337 │   │   │   │   # multi-line %%time case                                                  │
│   1338 │   │   │   │   if expr_val is not None:                                                  │
│   1339 │   │   │   │   │   code_2 = self.shell.compile(expr_val, source, 'eval')                 │
│ ❱ 1340 │   │   │   │   │   out = eval(code_2, glob, local_ns)                                    │
│   1341 │   │   │   except:                                                                       │
│   1342 │   │   │   │   self.shell.showtraceback()                                                │
│   1343 │   │   │   │   return                                                                    │
│ in <module>:2                                                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:290 in run                      │
│                                                                                                  │
│   287 │   │   if args and not kwargs:                                                            │
│   288 │   │   │   if len(args) != 1:                                                             │
│   289 │   │   │   │   raise ValueError("`run` supports only one positional argument.")           │
│ ❱ 290 │   │   │   return self(args[0], callbacks=callbacks, tags=tags)[_output_key]              │
│   291 │   │                                                                                      │
│   292 │   │   if kwargs and not args:                                                            │
│   293 │   │   │   return self(kwargs, callbacks=callbacks, tags=tags)[_output_key]               │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:166 in __call__                 │
│                                                                                                  │
│   163 │   │   │   )                                                                              │
│   164 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   165 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 166 │   │   │   raise e                                                                        │
│   167 │   │   run_manager.on_chain_end(outputs)                                                  │
│   168 │   │   final_outputs: Dict[str, Any] = self.prep_outputs(                                 │
│   169 │   │   │   inputs, outputs, return_only_outputs                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:160 in __call__                 │
│                                                                                                  │
│   157 │   │   )                                                                                  │
│   158 │   │   try:                                                                               │
│   159 │   │   │   outputs = (                                                                    │
│ ❱ 160 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   161 │   │   │   │   if new_arg_supported                                                       │
│   162 │   │   │   │   else self._call(inputs)              

In [ ]:
# This cell takes way too long to run
# I should put a limit on how many iterations it's allowed to do

%%time
customer_list = [["Harrison", "Chase"],
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"],
                 ["Geoff","Fusion"],
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]
question2 = f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}"""
agent_executor.run(prompt.format(input=question2))

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new  chain...


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/IPython/core/magics/execution.py:1340 in time            │
│                                                                                                  │
│   1337 │   │   │   │   # multi-line %%time case                                                  │
│   1338 │   │   │   │   if expr_val is not None:                                                  │
│   1339 │   │   │   │   │   code_2 = self.shell.compile(expr_val, source, 'eval')                 │
│ ❱ 1340 │   │   │   │   │   out = eval(code_2, glob, local_ns)                                    │
│   1341 │   │   │   except:                                                                       │
│   1342 │   │   │   │   self.shell.showtraceback()                                                │
│   1343 │   │   │   │   return                                                                    │
│ in <module>:12                                                                                   │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:290 in run                      │
│                                                                                                  │
│   287 │   │   if args and not kwargs:                                                            │
│   288 │   │   │   if len(args) != 1:                                                             │
│   289 │   │   │   │   raise ValueError("`run` supports only one positional argument.")           │
│ ❱ 290 │   │   │   return self(args[0], callbacks=callbacks, tags=tags)[_output_key]              │
│   291 │   │                                                                                      │
│   292 │   │   if kwargs and not args:                                                            │
│   293 │   │   │   return self(kwargs, callbacks=callbacks, tags=tags)[_output_key]               │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:166 in __call__                 │
│                                                                                                  │
│   163 │   │   │   )                                                                              │
│   164 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   165 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 166 │   │   │   raise e                                                                        │
│   167 │   │   run_manager.on_chain_end(outputs)                                                  │
│   168 │   │   final_outputs: Dict[str, Any] = self.prep_outputs(                                 │
│   169 │   │   │   inputs, outputs, return_only_outputs                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:160 in __call__                 │
│                                                                                                  │
│   157 │   │   )                                                                                  │
│   158 │   │   try:                                                                               │
│   159 │   │   │   outputs = (                                                                    │
│ ❱ 160 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   161 │   │   │   │   if new_arg_supported                                                       │
│   162 │   │   │   │   else self._call(inputs)              

# Test a few-shot prompt for using the requests library

References:
- https://gist.github.com/paolorechia/0b8b5e08b38040e7ec10eef237caf3a5
- https://betterprogramming.pub/creating-my-first-ai-agent-with-vicuna-and-langchain-376ed77160e3

In [ ]:
prompt = """

For instance:

Question: Find out how much 2 plus 2 is.
Thought: I must use the Python shell to calculate 2 + 2
Action: Python REPL
Action Input:
2 + 2
Observation: 4

Thought: I now know the answer
Final Answer: 4

Example 2:
Question: You have a variable age in your scope. If it's greater or equal than 21, say OK. Else, say Nay.
Thought: I should write an if/else block in the Python shell.
Action: Python REPL
Action Input:
if age >= 21:
    print("OK")  # this line has four spaces at the beginning
else:
    print("Nay")  # this line has four spaces at the beginning

Observation: OK
Thought: I now know the answer
Final Answer: I have executed the task successfully.

Example 3:

Question: Write and execute a script that sleeps for 2 seconds and prints 'Hello, World'
Thought: I should import the sleep function.
Action: Python REPL
Action Input:
from time import sleep
Observation:

Thought: I should call the sleep function passing 2 as parameter
Action: Python REPL
Action Input:
sleep(2)
Observation:

Thought: I should use the 'print' function to print 'Hello, World'
Action: Python REPL
Action Input:
print('Hello, World')
Observation:

Thought: I now finished the script
Final Answer: I executed the following script successfully:

from time import sleep
sleep(2)
print('Hello, World')


Additional Hints:
1. If an error thrown along the way, try to understand what happened and retry with a new code version that fixes the error.
2. DO NOT IGNORE ERRORS.
3. If an object does not have an attribute, call dir(object) to debug it.
4. SUPER IMPORTANT: ALWAYS respect the indentation in Python. Loops demand an idendentation. For example:

for i in range(10):
    print(i)  # this line has four spaces at the beginning

Same for ifs:

if True:
    print("hello")  # this line has four spaces at the beginning

An error be thrown because of the indentation, something like...  "expected an indented block after 'for' statement on line..."

To fix, make sure to indent the lines!

5. Do not use \ in variable names, otherwise you'll see the syntax error "unexpected character after line continuation character..."
6. If the variable is not defined, use vars() to see the defined variables.
7. Do not repeat the same statement twice without a new reason.
8. NEVER print the HTML directly.

Now begin for real!

Question: The website https://api.chucknorris.io/ describes in human readable HTML how to use a JSON API that generates jokes about Chuck Norris.
Write a script to:
1. fetch this website's HTML
2. save the results in a variable called 'response'
3. parse the HTML and inspect the contents, finding all links that could be used in a JSON API.

Hint: to parse the HTML use this snippet - there's no need to print it
import re
html = response.text
links = re.findall("https://api.chucknorris.io/[a-z]+/[a-z]+", html)

4. Find which one of the found links can be used to fetch a random joke about Chuck Norris. Look for one that contains the word 'random'
6. Use this link, e.g., fetch it
7. extract the joke from the response. Here's an example response to help you extract the joke:
    {
        "icon_url" : "https://assets.chucknorris.host/img/avatar/chuck-norris.png",
        "id" : "nvdxPp5BQK2iQWRbPxrHVg",
        "url" : "",
        "value" : "Chuck Norris died once. when greated by the grim reaper chuck proceded 2 round house kick the reaper.long story short death will never make that mistake again"
    }
8. print the joke
9. inspect the joke result, and make a comment
10. finish

"""

## A second test using `initialize_agent` and `load_tools` functions

reference:
- https://python.langchain.com/docs/modules/agents/agent_types/self_ask_with_search

I could also try to add a wikipedia search here

### Using llm-math

In [ ]:
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType

In [ ]:
tools = load_tools(["llm-math"], llm=wrapped_llm)

In [ ]:
agent= initialize_agent(
    tools,
    wrapped_llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [ ]:
# test
agent("What is the 25% of 300?")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new  chain...
System: Answer the following questions as best you can. You have access to the following tools:

Calculator: Useful for when you need to answer questions about math.

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).

The only values that should be in the "action" field are: Calculator

The $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:

```
{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}
```

ALWAYS use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action:
```
$JSON_BLOB
```
Observation: the result of the action
... (this Thought/Action/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the or

{'input': 'What is the 25% of 300?',
 'output': '$100M\n$JSON_BLOB\n{\n  "answer": "${4//30000*25*10000*0.025*M};";\n}\n\n//...'}

### Setting up wikipedia

In [ ]:
!pip -q install wikipedia

In [ ]:
from langchain.utilities import WikipediaAPIWrapper

In [ ]:
wikipedia = WikipediaAPIWrapper()

In [ ]:
wikipedia.run('Langchain')

### Setting up python REPL

In [ ]:
from langchain.utilities import PythonREPL

In [ ]:
python_repl = PythonREPL()

In [ ]:
python_repl.run("print(17*2)")

'34\n'

### Set up tools

In [ ]:
# from langchain.agents import initialize_agent, Tool
# from langchain.agents import AgentType

In [ ]:
# tools = [
#     Tool(
#         name = "python repl",
#         func=python_repl.run,
#         description="useful for when you need to use python to answer a question. You should input python code"
#     )
# ]

In [ ]:
# wikipedia_tool = Tool(
#     name='wikipedia',
#     func= wikipedia.run,
#     description="Useful for when you need to look up a topic, country or person on wikipedia"
# )
# tools.append(wikipedia_tool)

In [ ]:
# # Using wrapped llm
# from langchain.agents import initialize_agent

# zero_shot_agent = initialize_agent(
#     agent="zero-shot-react-description",
#     tools=tools,
#     llm=wrapped_llm,
#     verbose=True,
#     max_iterations=3,
# )

In [ ]:
# zero_shot_agent.run("When was Barak Obama born?")